# Combining templates and plotting systematic shifts

Author(s): Raghav Kansal, Javier Duarte

In [1]:
import pickle
import json
from pathlib import Path
import hist
from hist import Hist
import warnings
import numpy as np


from HH4b.hh_vars import jecs, jmsr, bg_keys, sig_keys, LUMI, years
from HH4b import utils, plotting, postprocessing
from HH4b.postprocessing import datacardHelpers, Region, get_weight_shifts

weight_shifts = get_weight_shifts("glopart-v2", "24Nov7_v5_glopartv2_rawmass")

In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
templates_path = Path(
    "/home/users/woodson/HH4b/src/HH4b/postprocessing/templates/25Feb27GloParTv2BDTv13LoosenTXbb"
)
templates_path_original = Path(
    "/home/users/woodson/HH4b/src/HH4b/postprocessing/templates/24Nov11GloParTv2ZichunPRNoTTbarSF"
)

## Combining templates

In [12]:
templates = {}
templates_original = {}

years = ["2022", "2022EE", "2023", "2023BPix"]
for year in years:
    with (templates_path / f"{year}_templates.pkl").open("rb") as f:
        templates[year] = pickle.load(f)

    with (templates_path_original / f"{year}_templates.pkl").open("rb") as f:
        templates_original[year] = pickle.load(f)

## Testing JES plots

In [15]:
main_plot_dir = Path("plots/PostProcessing/25Feb27GloParTv2BDTv13LoosenTXbb/Templates")


selection_regions = {
    "pass_bin1": Region(
        cuts={
            "Category": [1, 2],
        },
        label="Bin1",
    ),
    "pass_bin2": Region(
        cuts={
            "Category": [2, 3],
        },
        label="Bin2",
    ),
    "pass_bin3": Region(
        cuts={
            "Category": [3, 4],
        },
        label="Bin3",
    ),
    "fail": Region(
        cuts={
            "Category": [4, 5],
        },
        label="Fail",
    ),
}

In [16]:
def combine_hists(*hists):
    csamples = []
    for h in hists:
        csamples += list(h.axes[0])

    reth = Hist(
        hist.axis.StrCategory(csamples, name="Sample"),
        *hists[0].axes[1:],
        storage="weight",
    )

    for h in hists:
        for sample in h.axes[0]:
            reth.view(flow=True)[utils.get_key_index(reth, sample), ...] = h[sample, ...].view(
                flow=True
            )

    return reth

In [17]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))

# this is needed for some reason to update the font size for the first plot
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plt.rcParams.update({"font.size": 24})
plt.close()

In [19]:
warnings.filterwarnings("ignore")

sig_key = "hh4b"
# sig_key = "vbfhh4b-k2v0"
# sig_key = "ttbar"
# sig_key = "vbfhh4b-kvm0p962-k2v0p959-klm1p43"
# sig_key = "diboson"

for rname, region in selection_regions.items():
    if rname != "pass_bin1":
        continue
    plot_dir = main_plot_dir / "2022-2023" / "wshifts"
    plot_dir.mkdir(exist_ok=True, parents=True)
    # for wshift in list((jecs).keys()):
    for wshift in [
        # "ttbarSF_Xbb_bin_0_0.8",
        # "ttbarSF_Xbb_bin_0.8_0.94",
        # "ttbarSF_Xbb_bin_0.94_0.99",
        # "ttbarSF_Xbb_bin_0.99_1",
        # "ttbarSF_Xbb_bin_0_0.31",
        # "ttbarSF_Xbb_bin_0.31_0.7",
        # "ttbarSF_Xbb_bin_0.7_0.8",
        # "ttbarSF_Xbb_bin_0.8_0.87",
        # "ttbarSF_Xbb_bin_0.87_0.92",
        # "ttbarSF_Xbb_bin_0.92_0.96",
        # "ttbarSF_Xbb_bin_0.96_1",
        # "ttbarSF_pTjj",
        # "ttbarSF_tau32",
        # "ttbarSF_BDT_bin_0.03_0.3",
        # "ttbarSF_BDT_bin_0.3_0.5",
        # "ttbarSF_BDT_bin_0.5_0.7",
        # "ttbarSF_BDT_bin_0.7_0.93",
        # "ttbarSF_BDT_bin_0.93_1.0",
        # "trigger"
        # "TXbbSF_correlated",
        "TXbbSF_uncorrelated_WP1_pT_bin_250_300",
        "TXbbSF_uncorrelated_WP1_pT_bin_300_400",
        "TXbbSF_uncorrelated_WP1_pT_bin_400_500",
        "TXbbSF_uncorrelated_WP1_pT_bin_500_100000",
        # "JMR",
        # "JMS",
    ]:
        if wshift in jecs or wshift in jmsr:
            # adding jshift-ed histograms into the same histogram
            cjshift_templates = [sum([templates[year][rname] for year in years])]
            for shift in ["up", "down"]:
                # new histogram with sample names renamed to "{sample}_{jsf}_{shift}"
                jt = sum([templates[year][f"{rname}_{wshift}_{shift}"] for year in years])
                slabels = [f"{s}_{wshift}_{shift}" for s in jt.axes[0]]
                rjt = Hist(
                    hist.axis.StrCategory(slabels, name="Sample"),
                    *jt.axes[1:],
                    storage="weight",
                )
                rjt.view()[...] = jt.view()
                cjshift_templates.append(rjt)

                cjt = combine_hists(*cjshift_templates)
                shift_label = wshift
        else:
            print(weight_shifts.keys())
            shift_label = weight_shifts[wshift].label
            cjt = sum([templates[year][rname] for year in years])
        print(cjt)

        plotting.sigErrRatioPlot(
            cjt,
            sig_key,
            wshift,
            "$m_{reg}^{2}$ (GeV)",
            f"{region.label} Region {shift_label} Variations",
            plot_dir,
            f"{rname}_sig_{wshift}",
            show=True,
            ylim=[0, 2.5],
            h_uncorr=sum([templates_original[year][rname] for year in years]),
        )

dict_keys(['ttbarSF_pTjj', 'ttbarSF_tau32', 'trigger', 'TXbbSF_correlated', 'ttbarSF_Xbb_bin_0_0.31', 'ttbarSF_Xbb_bin_0.31_0.7', 'ttbarSF_Xbb_bin_0.7_0.8', 'ttbarSF_Xbb_bin_0.8_0.87', 'ttbarSF_Xbb_bin_0.87_0.92', 'ttbarSF_Xbb_bin_0.92_0.96', 'ttbarSF_Xbb_bin_0.96_1', 'ttbarSF_ggF_BDT_bin_0.03_0.6375', 'ttbarSF_ggF_BDT_bin_0.6375_0.9075', 'ttbarSF_ggF_BDT_bin_0.9075_1.0', 'ttbarSF_VBF_BDT_bin_0.975_1', 'TXbbSF_uncorrelated_WP5_pT_bin_200_400', 'TXbbSF_uncorrelated_WP4_pT_bin_200_400', 'TXbbSF_uncorrelated_WP3_pT_bin_200_400', 'TXbbSF_uncorrelated_WP2_pT_bin_200_400', 'TXbbSF_uncorrelated_WP1_pT_bin_200_400'])


KeyError: 'TXbbSF_uncorrelated_WP1_pT_bin_250_300'

In [ ]:
from rhalphalib import MorphHistW2
from HH4b.postprocessing.datacardHelpers import smass, smorph
from HH4b.hh_vars import jmsr_values

jms_values = jmsr_values["JMS"]
jmr_values = jmsr_values["JMR"]

In [ ]:
templ = templates_original["pass_bin1"]["hh4b", :]
templ_nominal = smorph(templ, "hh4b", jms_values[year]["nom"], jmr_values[year]["nom"])
templ_jms_dn = smorph(templ, "hh4b", jms_values[year]["down"], jmr_values[year]["nom"])
templ_jms_up = smorph(templ, "hh4b", jms_values[year]["up"], jmr_values[year]["nom"])
templ_jmr_dn = smorph(templ, "hh4b", jms_values[year]["nom"], jmr_values[year]["down"])
templ_jmr_up = smorph(templ, "hh4b", jms_values[year]["nom"], jmr_values[year]["up"])

In [ ]:
templ = templates_original["pass_bin1"]["hh4b", :]
templ_nominal = templates["pass_bin1"]["hh4b", :]
templ_jms_dn = templates["pass_bin1_JMS_down"]["hh4b", :]
templ_jms_up = templates["pass_bin1_JMS_up"]["hh4b", :]
templ_jmr_dn = templates["pass_bin1_JMR_down"]["hh4b", :]
templ_jmr_up = templates["pass_bin1_JMR_up"]["hh4b", :]

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np

edges = templ.axes[0].edges
var_name = templ.axes[0].name
ymax = np.max(templ.values()) * 1.4

plt.figure(figsize=(21, 7))
ax1 = plt.subplot(131)
hep.histplot(
    templ.values(),
    edges,
    yerr=np.sqrt(templ.variances()),
    histtype="step",
    label="Original (JMS=1, JMR=1)",
)
hep.histplot(
    templ_nominal.values(),
    edges,
    yerr=np.sqrt(templ_nominal.variances()),
    histtype="step",
    label=f"Nominal (JMS={jms_values[year]['nom']}, JMR={jmr_values[year]['nom']})",
)
ax2 = plt.subplot(132, sharey=ax1)
hep.histplot(
    templ_jms_dn.values(),
    edges,
    yerr=np.sqrt(templ_jms_dn.variances()),
    histtype="step",
    label=f"JMS Down (JMS={jms_values[year]['down']})",
)
hep.histplot(
    templ_jms_up.values(),
    edges,
    yerr=np.sqrt(templ_jms_up.variances()),
    histtype="step",
    label=f"JMS Up (JMS={jms_values[year]['up']})",
)
ax3 = plt.subplot(133, sharey=ax1)
hep.histplot(
    templ_jmr_dn.values(),
    edges,
    yerr=np.sqrt(templ_jmr_dn.variances()),
    histtype="step",
    label=f"JMR Down (JMR={jmr_values[year]['down']})",
)
hep.histplot(
    templ_jmr_up.values(),
    edges,
    yerr=np.sqrt(templ_jmr_up.variances()),
    histtype="step",
    label=f"JMR Up (JMR={jmr_values[year]['up']})",
)

for ax in [ax1, ax2, ax3]:
    ax.legend(fontsize=18)
    ax.set_ylim(0, ymax)
    ax.set_xlim(60, 220)
    ax.set_xlabel("$m^{j2}_{reg}$ [GeV]")

ax1.set_ylabel("Signal Yield")
ax2.set_title(f"ggF HH4b ({year})")
plt.savefig(main_plot_dir / f"hh4b_jms_jmr_{year}.pdf")
plt.show()